In [ ]:
run_name   = "s_delta_set24"
# data_list  = ["parkinsons","vertebral","breast","climate", "ionosphere", "blod", "bank", "QSAR", "spambase"] 
data_list = ["parkinsons","vertebral"]

In [3]:
import res_AUROC as roc

for data in data_list:
    query = f"SELECT results, id , prams, result_type FROM experiments Where task='unc' AND dataset='Jdata/{data}' AND run_name='{run_name}'"
    # query = f"SELECT results, id , prams, result_type FROM experiments Where task='unc' AND id=5604 OR id=5712 OR id=5703"

    res = roc.create_roc_table(data_list, query, "epsilon")
    print(f"-------------------------------------------------------------------------------------- {data}")
    display(res[0].style.apply(roc.highlight_max_min))

-------------------------------------------------------------------------------------- parkinsons


,Method,Parameter,Epistemic,e_sd,Aleatoric,a_sd,Total,t_sd
job_id,,,,,,,,
5655,set24,1.010000,0.855099,0.091224,0.845717,0.138058,0.845520,0.136236
5664,set24,1.050000,0.855250,0.089501,0.844003,0.136847,0.847626,0.136332
5673,set24,1.100000,0.854900,0.089988,0.843682,0.138031,0.850574,0.137992
5682,set24,1.200000,0.855910,0.091384,0.835137,0.148002,0.853476,0.133376
5691,set24,1.300000,0.856430,0.091557,0.818497,0.153751,0.854826,0.137246
5700,bays,1.300000,0.845516,0.112065,0.754000,0.202789,0.848729,0.142685


-------------------------------------------------------------------------------------- vertebral


,Method,Parameter,Epistemic,e_sd,Aleatoric,a_sd,Total,t_sd
job_id,,,,,,,,
5656,set24,1.010000,0.809552,0.109991,0.814693,0.111343,0.814261,0.112019
5665,set24,1.050000,0.809604,0.109907,0.816369,0.112861,0.814994,0.113313
5674,set24,1.100000,0.809328,0.109854,0.819794,0.111300,0.814741,0.112818
5683,set24,1.200000,0.809516,0.109072,0.823148,0.115201,0.816819,0.113086
5692,set24,1.300000,0.809852,0.106536,0.822268,0.124001,0.817501,0.123801
5701,bays,1.300000,0.776564,0.125634,0.810846,0.112037,0.820515,0.124113
